# 関数型プログラミング

## 関数型プログラミングの特徴

### ファーストクラスオブジェクトとしての関数
- 関数型プログラミングでは、関数はファーストクラスオブジェクトとして扱われる
    - ファーストクラスオブジェクト:
        - 計算の対象となる値
        - 変数にバインドしたり、データ構造に埋め込んだり、関数の引数・返戻値とすることが可能
- 関数がファーストクラスオブジェクトであることの最大の利点は、高階関数が利用可能になること
    - **高階関数**: 引数・返戻値に関数を指定することが可能な関数

### 参照透過性
- 参照透過性: 「プログラムの構成要素が同じもの同士は等しい」という性質
    - 記号表現: `A == A`
        - 左辺・右辺の記号 `A` が何を示しているとしても、それらは必ず等しくなければならない
    - 例:
        - `2 == 2`: 2 は 2 に等しい
        - `1 + 1 == 2`: 1+1 の計算結果は 2 に等しい
- 参照透過性が成立する条件:
    - 通常、同じ値・変数は必ず参照透過性が成立する
        - `2 == 2`
        - `x = 100; x == x`
    - 関数については、「関数の呼び出し結果が、その関数の引数のみに依存する」場合のみ参照透過性が成立
        - 逆説的に、「関数の呼び出し結果が、その関数の引数のみに依存しない」場合は参照透過性が成立しない
        - 参照透過性が成立しない関数を **副作用** のある関数と呼ぶ
        - 特に、可変なデータを引数にとって、そのデータの内容を変更してしまうような関数を **破壊的メソッド** と呼ぶ

In [4]:
"""
push!() 関数: 可変なデータ構造を変更するため、副作用のある関数 => 参照透過性が成立しない

* Julia において、破壊的メソッドは慣例的に `!` を後ろにつけた関数名としている
"""
# Vector: 可変なデータ構造
vec = [1]

# push!(vec, 2) 関数の返戻値をコピーして持っておく
x = push!(vec, 2) |> copy

# push!(vec, 2) 関数の返戻値をコピーして持っておく
y = push!(vec, 2) |> copy

# 参照透過性が成立している場合、上記 x, y は等しくなければならない
display((x, y))
display(x == y)

([1, 2], [1, 2, 2])

false

In [3]:
"""
vcat() 関数: 可変なデータ構造を変更しないため、副作用のない関数 => 参照透過性が成立する
"""
# Vector: 可変なデータ構造
vec = [1]

# vcat(vec, 2) 関数の返戻値をコピーして持っておく
x = vcat(vec, 2) |> copy

# vcat(vec, 2) 関数の返戻値をコピーして持っておく
y = vcat(vec, 2) |> copy

# 参照透過性が成立している場合、上記 x, y は等しくなければならない
display((x, y))
display(x == y)

([1, 2], [1, 2])

true

### 参照透過性の保証
参照透過性が保証されていない場合、処理結果の予測が難しくなるため、コードの正しさを見た目から判断することができなくなる

これがソフトウェアのバグを発生させる一番の原因である

- 参照透過性を喪失するケース:
    1. 値の参照透過性を破壊する可変なデータ
        - 可変なデータはすべて参照透過性を喪失する可能性をはらんでいる
    2. 変数の参照透過性を破壊する代入操作
        - 値の再代入可能な変数はすべて参照透過性を喪失する可能性をはらんでいる
    3. 関数の参照透過性を破壊する副作用
        - 破壊的メソッドは参照透過性を喪失する
            - `push!()` 等の可変データ操作関数
        - 副作用のある関数は参照透過性を保証しない
            - `Dates.now()` 等の関数は、そもそも返戻値が時間に依存するため参照透過性を保証しない

上記3つのケースを回避することで、コードの参照透過性を保証することができる

#### 値の参照透過性: 不変なデータ
値の参照透過性を保証するためには、可変なデータ構造を利用しなければ良い

一度生成されたデータの中身が変更されないデータのことを **不変なデータ** と呼ぶ

In [7]:
"""
不変なデータ構造の定義
"""

"""
    empty() ::Nothing

普遍的に nothing なデータ
"""
empty() ::Nothing = nothing

"""
    struct ConstData{T} end

中身の値が普遍的に {T} であることが保証されたデータ
"""
struct ConstData{T} end

get(::ConstData{T}) where T = T
set(value::Any) = ConstData{value}()

display(empty() === nothing)
display(get(set(100)) === 100)

true

true

#### 変数の参照透過性: 代入の抑制
変数は代入によって参照透過性を喪失するため、変数の参照透過性を保証するためには代入を行わなければ良い

ここで、多くのプログラミング言語では「束縛」も「代入」も同じ `=` 演算子を使っているが、この2つは明確に別の操作である

「束縛」は「値に変数名というラベル付けを行う」ことであり、「代入」は「変数に値を設定する」ことである

言い換えると、「束縛」では「一つの変数ラベルには一つの値が対応する」のに対して、「代入」では「一つの変数ラベルに一つ以上の値が対応し得る」ことになる

最近のプログラミング言語では、値の束縛を行うために `const` 等のキーワード宣言が行えるようになっており、こういった変数に対しては代入ができないように言語レベルでサポートされていることが多い

In [9]:
"""
足し算の定義: 命令型モデルと関数型モデルの比較
"""
# 反復文を用いた足し算の定義 (命令型モデル)
## 代入操作が内部で行われていて、参照透過性が保証されていない
addInstruction(x::Int, y::Int) ::Int = begin
    result = x
    for times = 1:y # times 変数への代入操作
        result = result + 1 # result 変数への代入操作
    end
    result
end

# 関数型プログラミングによる足し算の定義
## 繰り返し処理を再帰呼び出しで代替することで、代入操作を不要にし、参照透過性を保証している
addFunction(x::Int, y::Int) ::Int = y < 1 ? x : addFunction(x + 1, y - 1)

# 4 + 6 -> 10
display(addInstruction(4, 6))
display(addFunction(4, 6))

10

10

#### 関数の参照透過性: 副作用の分離
関数の参照透過性を保証するには、副作用を排除すれば良い

しかしながら、すべての副作用を排除して良いかと言えば、話はそう単純ではない

計算機には、計算結果を人間に表示する機能や、データを外部の記憶装置に蓄積する機能などが不可欠だからである

そのため、副作用を制御するための基本的な戦略は、副作用を持つ部分と持たない部分を分離することになる

両者をきちんと分離することで、少なくとも副作用のない **純粋な関数** については、参照透過性の利点を享受できるようになる

このとき、純粋な関数から副作用のある関数を呼び出してしまうと、副作用汚染が発生し、参照透過性が失われてしまうため注意する必要がある

In [10]:
using Dates

"""
    getAge(birthYear::Int) ::Int

年齢を計算する（副作用が分離されていない関数）
* 関数実行時の年が変わると実行結果が変わってしまう
"""
getAge(birthYear::Int) ::Int = year(now()) - birthYear # 副作用のある now() 関数を呼んでしまっている

"""
    getAge(birthYear::Int, thisYear::Int) ::Int

年齢を計算する（副作用が分離されている関数）
* 処理結果は純粋に関数の引数のみに依存する
"""
getAge(birthYear::Int, thisYear::Int) ::Int = thisYear - birthYear

getAge(2000) |> display
getAge(2000, year(now())) |> display

22

22

## 関数型プログラミングの利点

### モジュール性が高い
ソフトウェアに無秩序に機能を追加していくと、そのコードはすぐに巨大で複雑なものへと膨れ上がる

この問題を回避する有効な手段は、全体を小さな単位に分割するという戦略である

その小さな単位を **モジュール** と呼び、問題をモジュールに分割することをモジュール化と呼ぶ

モジュール化によるプログラミングは、次のような手順で問題を解決する

1. 大きな問題を小さな問題に分割する
2. 小さな部品で小さな問題を解く
3. 小さな問題を解く小さな部品を組み合わせて、大きな問題を解く

関数型プログラミングにおいて、小さな部品（モジュール）に相当するのが「値（データ）」「変数」「関数」であり、これらの小さな部品を繋ぎ合わせるのが関数の役目となる

モジュール化を成功させるためには、以下の条件を満たす必要がある

- 部品の独立性
    - 個々のモジュールは、他のモジュールの機能の影響を受けてはならない（参照透過性が保証されている必要がある）
    - コードがモジュールとして独立していれば、特定のコードの変更が他の箇所への影響する可能性を軽減し、バグの発生を抑制することができる
- 部品の汎用性
    - 各モジュールは複数の箇所で利用できることを前提に設計する必要がある
    - モジュールが汎用的であるためには、モジュール同士を繋げる接続部分 (**インターフェイス**) が単純で統一的である必要がある
- 部品の合成可能性
    - モジュールは組み合わせることで大きな問題を解くため、モジュール同士を合成できる必要がある
    - 部品の汎用性とも被るが、モジュール同士のインターフェイスが単純で統一的であるほど、合成しやすくなる
    - 関数型プログラミングでは、関数を用いてモジュールを合成するため、関数の引数・返戻値が統一的であることが重要となる


In [17]:
###################################
# 部品の独立性: 参照透過性を保証する #
###################################
"""
    getAge(birthYear::Int, thisYear::Int) ::Int

年齢算出関数（副作用のない関数）
"""
getAge(birthYear::Int, thisYear::Int) ::Int = thisYear - birthYear

using Dates
getAge(2000, year(now())) |> display


###########################################
#               部品の汎用性               #
# - 関数の引数・返戻値をなるべく統一する     #
# - 個々のモジュールはなるべく小さく設計する #
#   - 一つのモジュールは一つの問題を解くべき #
# - 関数渡しを適切に利用する                #
###########################################
"""
    reduceVector(callback::Function, target::Vector{T}, accumulator::T) ::T where T

配列に対して関数を適用し、一つの値に蓄積する汎用関数

- `callback::Function`: `(accumulator::Any, item::T) ::T where T` 適用関数
    - `accumulator::T`: 計算結果が蓄積された値
    - `item::T`: target[x] (配列の各要素の値)
    - `@returns::T`: 計算結果の値 => accumulator に蓄積される
- `taget::Vector{T}`: 関数適用対象の配列
- `accumulator::T`: 計算結果を蓄積する値の初期値
"""
reduceVector(callback::Function, target::Vector{T}, accumulator::T) where T =
    length(target) > 1 ?
        reduceVector(callback, target[2:end], callback(accumulator, target[1])) :
        (isempty(target) ? accumulator : callback(accumulator, target[1]))

"""
    sum(target::Vector{T}, accumulator::T) ::T where T

配列の要素をすべて加算し合計を算出する
* reduceVector 汎用関数を利用することで簡単に定義できる
"""
sum(target::Vector{T}, accumulator::T) where T = reduceVector((acc, item) -> acc + item, target, accumulator)

"""
    product(target::Vector{T}, accumulator::T) ::T where T

配列の要素をすべて乗算した結果を算出する
* reduceVector 汎用関数を利用することで簡単に定義できる
"""
product(target::Vector{T}, accumulator::T) where T = reduceVector((acc, item) -> acc * item, target, accumulator)

sum([1, 2, 3, 4, 5], 0) |> display
product([1, 2, 3, 4, 5], 1) |> display

##########################################################################
#                            部品の合成可能性                             #
# - 関数型プログラミングでは、関数同士を繋ぎ合わせるため **関数の合成** と呼ぶ #
# - 遅延評価を利用することで関数のモジュール性をさらに向上させることができる   #
#   - 遅延評価: コードの一部の計算を即時処理せず、後回しにすること            #
##########################################################################
"""
    compose(f::Function, g::Function) ::Function

2つの関数を合成し、処理を繋ぎ合わせる: g() => f()
"""
compose(f::Function, g::Function) ::Function = (arg::Any) -> f(g(arg))

"""
    sequence(n::Int) ::Vector{Int} = [n, n+1, n+2, ...]

無限数列の実装
* 遅延評価によるストリーム型を利用して、無限に続くデータ構造を表現できる
* ストリーム型: 値の並びの一部を遅延評価したデータ構造
"""
sequence(n::Int) ::Vector = [
    n,
    # 2つ目以降の数値を関数化することで遅延評価
    # 再帰呼び出しすることで無限に遅延評価されるようになる
    () -> sequence(n + 1)
]

display(sequence(1))
display(sequence(1)[2]())
display(sequence(1)[2]()[2]())

22

15

120

2-element Vector{Any}:
 1
  #57 (generic function with 1 method)

2-element Vector{Any}:
 2
  #57 (generic function with 1 method)

2-element Vector{Any}:
 3
  #57 (generic function with 1 method)

### テストが容易
モジュール化によりコードの独立性が高まると、単体テストが容易になる

参照透過性のあるコードはその依存関係が明確であり、テストを何度繰り返しても結果は不変であるためである

単体テストを行うことで、少なくともテストを行った項目に対して正常に処理が行われることが証明できるため、バグの発生を抑制することができる

In [19]:
"""
    mutable struct Player end

ゲームプレイヤー

# Arguments

- `name::AbstractString`: プレイヤー名
- `score::Int`: 点数
"""
mutable struct Player
    name::AbstractString
    score::Int
end


###############################################
# 単体テストを行いにくいコード = 副作用を持つ関数 #
###############################################
"""
    displayWinner(player1::Player, player2::Player)

点数の多い方のプレイヤーの名前をコンソールに表示する
"""
displayWinner(player1::Player, player2::Player) = begin
    if player1.score > player2.score
        println("$(player1.name)が勝者")
    elseif player1.score < player2.score
        println("$(player2.name)が勝者")
    else
        println("引き分け")
    end
end

# 副作用を持つ関数は、目視でテストを行うしかない
displayWinner(Player("甲", 10), Player("乙", 12))
displayWinner(Player("甲", 10), Player("乙", 9))
displayWinner(Player("甲", 5), Player("乙", 5))

乙が勝者
甲が勝者
引き分け


In [20]:
# MaybePlayer = Player | Nothing
MaybePlayer = Union{Player, Nothing}

#######################################
# 副作用のある関数と純粋な関数に分離する #
#######################################
"""
    getWinner(player1::Player, player2::Player) = winner::MaybePlayer

プレイヤー同士のスコアを比較し、勝者を取得する (引き分けの場合は nothing を返す)
* 純粋な関数のため、単体テストしやすい
"""
getWinner(player1::Player, player2::Player) ::MaybePlayer = begin
    player1.score > player2.score && return player1
    player1.score < player2.score && return player2
    nothing # 引き分けの場合は nothing を返す
end

"""
    makeAnnounce(winner::MaybePlayer) = announce::AbstractString

勝者を告げる文字列を生成する
* getWinner() と繋ぎやすくするため、getWinner() の返戻型と makeAnnounce() の引数型を統一している
* 純粋な関数のため、単体テストしやすい
"""
makeAnnounce(winner::MaybePlayer) ::AbstractString = isnothing(winner) ? "引き分け" : "$(winner.name)が勝者"

"""
    displayWinner(player1::Player, player2::Player)

点数の多い方のプレイヤーの名前をコンソールに表示する
* 副作用のある関数: 目視でテストするしかない
* 内部処理は純粋な関数の組み合わせでできているため、純粋な関数の妥当性検査さえできていれば、この関数自体の妥当性を検査する必要はあまりない
  - 内部処理: 純粋関数`getWinner` |> 純粋関数`makeAnnounce` |> 副作用関数`plintln`
"""
displayWinner(player1::Player, player2::Player) = getWinner(player1, player2) |> makeAnnounce |> println

displayWinner(Player("甲", 10), Player("乙", 12))
displayWinner(Player("甲", 10), Player("乙", 9))
displayWinner(Player("甲", 5), Player("乙", 5))

乙が勝者
甲が勝者
引き分け


In [23]:
"""
単体テスト
"""
# Julia では単体テスト用に Test.jl パッケージが用意されている
using Test

@testset "getWinner() 関数の単体テスト" begin
    # score:10 < score:12 のため player2 が期待値
    player1, player2 = Player("甲", 10), Player("乙", 12)
    @test getWinner(player1, player2) === player2

    # score:10 > score:9 のため player1 が期待値
    player1, player2 = Player("甲", 10), Player("乙", 9)
    @test getWinner(player1, player2) === player1

    # score:5 == score:5 のため nothing が期待値
    player1, player2 = Player("甲", 5), Player("乙", 5)
    @test getWinner(player1, player2) === nothing
end

@testset "makeAnnounce() 関数の単体テスト" begin
    # 引数が Player の場合は "$(player.name)が勝者" が期待値
    @test makeAnnounce(Player("丙", 10)) === "丙が勝者"

    # 引数が Nothing の場合は "引き分け" が期待値
    @test makeAnnounce(nothing) === "引き分け"
end

Test Summary:        | Pass  Total
getWinner() 関数の単体テスト |    3      3
Test Summary:           | Pass  Total
makeAnnounce() 関数の単体テスト |    2      2


Test.DefaultTestSet("makeAnnounce() 関数の単体テスト", Any[], 2, false, false)

### コードの正しさを証明できる
参照透過性は `A == A` という等式が成立することであるため、等式によるコードの **推論** が可能である

- 推論: ある式をもとにしてそれと等しい式を導出すること
    - 言い換えれば、定義済みの置換ルールを適用することで、同等の意味を持つ別の式に変形すること
- 単体テストも推論を使ったコードの正しさの証明と言える
    - `関数() == 期待値` が成立するなら、その関数は正しいと言える